# READ - 2

**This is just the testing part of the model that we created in Experimental.ipynb**

we can avoid using tensorflow instead we should prefer just using keras , if you want to deploy your model to online platform such as heroku. Because tensorflow uses a lot of memory and heroku one dyno (free ) version provides only 512 mb of RAM. Believe me it was such a pain in the ... :-) . Tensorflow 2.2.0 has installation size of approx 459 mb which almost takes 95% of your space. Heroku has this compressing facility which creates slugs after compression even though my slug size was 650 mb due to this the build (deployment) failed. 


---


SOLUTION :- what i did is degraded my tensorflow version to 2.0.1 in      requirements.txt even though my backend was tf2.2.0 it worked.

i have attached 'requirements.txt' in [GIT REPO](https://github.com/cd-x/test-pdd)  there you can see it is quite a big list of requirements. By the way you create reuirements.txt just by using command 

```
$pip install freeze
$pip freeze > requirements.txt
```
we'll discuss more on deployment in another notebook

# Code Explanation

TO load the trained model saved as 'acc9275own.h5' (h5 is the extension for model weights ) we need to import load_model from keras.models 


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

An interesting thing i found during loading the model is i was getting error as FILE NOT FOUND: no model found at /path/to/model { save_model.pb | save_model.txt } 

it was because as i have explained in experimental.ipynb that i was using **tensorflow_hub** to import pretrained feature vector but it doesn't get combined with the rest of the layers so the model didn't build succesfully. 

what colab does is it hides all the tedious task from you and create tfhub_modules in /tmp directory of your mounted drive

To reload model trained through **tensorflow_hub** what you need to do is 
save the model as follows

```
import time
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
tf.keras.models.save_model(model,export_path,save_format='h5')
```
It stores the model in /tmp/saved_model/ named as timestamp file and the directory will look like this

---tmp/

-----/saved_models

--------1234567

-----/tfhub_modules

-------/assets

-------/variables

----------variable index

----------variable.0000 - 0001

-------saved_model.pb



to reload the model the code will be as follows 



```
model = tf.keras.models.load_model(export_path, custom_objects={'KerasLayer':hub.KerasLayer})
```
NOTE 1:- even though training through tf_hub was easy with same accuracy as of now , it lead me to failure in deployment 

NOTE 2:- you can create above mentioned directory in another way by saving the model as **"model.h5py"** file .

**Remember i discarded the above approach for the sake of heroku**


In [ ]:
model=load_model('acc9275own.h5')

# LOADING categories.json

you can find the file in git repo it has all the categories mentioned in json dictionary so i am getting all the values of the dictionary and storing as list because dictioanry is not iterable in python. 

In [ ]:
import json

with open('categories.json', 'r') as f:
    cat_to_name = json.load(f)
    classes = list(cat_to_name.values())
    
print (classes)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

This image size  is fixed because our model has given input of 224 X 224 changing configuration will lead in model error

In [ ]:
IMAGE_SIZE=(224,224)


***load_image():*** method preprocess the given image using opencv_python (compuetr vision loads an image and resize in the input format of our sequential model)

***predict():*** method passes input tensor (created by numpy) to the model and will call predict method of keras.models. predict returns confidence of each 38 classes. Using numpy's argmax will return the index of max value comes first in the array for e.g. arr = 5 1 2 3 4 5 then argmax(arr) = 0.

After getting index we'll return category name present in the classes variable ,that we obtained previously from json


In [ ]:
# Import OpenCV
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob

def load_image(filename):
    img = cv2.imread(filename)
    #img = cv2.imread(os.path.join(image_dir, filename)) #<-- use in case of test through existing validation dataset
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img


def predict(image):
    probabilities = model.predict(np.asarray([image]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

# Upload image from anywhere and test it

having fun colab features

In [ ]:

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img= load_image(path)
  result= predict(img)
  print(result)